In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
import h5py
# from matplotlib.patches import StepPatch

cut_lens = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30, 35, 40, 50, 60, 70, 80, 90, 100, 110, 120]
test_cut_lens = [120]
desired_shift = 3
max_shift = desired_shift * 2

In [2]:
### ----- Load the MLAAPDE testing dataset from HDF5 files ----- ###

hdf5_save_dir = '/hdd/mlaapde/decimated'

testing_data = h5py.File(hdf5_save_dir + '/testing_data.hdf5', 'r')
test_waves = testing_data['waves'][:]
test_mags = testing_data['magnitude'][:]
test_phase_id = testing_data['phase_id'][:]

testing_data.close()

In [19]:
phase_id = str(test_phase_id[22647])
phase_id

"b'us60007fjg_AK.UNV.BH*.--_Pn'"

In [21]:
event_id = phase_id.split("'")[1].split('.')[0].split('_')[0]
event_id

'us60007fjg'

In [26]:
mlaa_eventids = []

for i in range(len(test_phase_id)):
    
    pid = str(test_phase_id[i])
    eventid = pid.split("'")[1].split('.')[0].split('_')[0]
#     print(eventid)
    mlaa_eventids.append(eventid)

In [28]:
len(mlaa_eventids)

324365

In [43]:
# unique_mlaaevents = np.unique(np.array(mlaa_eventids))
# # us6000c9hg

# for i in range(len(unique_mlaaevents)):
#     print(unique_mlaaevents[i])

In [29]:
### ----- Load the historic testing dataset ----- ###

historic_path = '/hdd/mlaapde/decimated/'

historic_test_waves = np.load(historic_path + 'historic_test_waves.npy')
historic_test_mags = np.load(historic_path + 'historic_test_mags.npy')
historic_test_eventids = np.load(historic_path + 'historic_test_eventids.npy')

print(historic_test_waves.shape)
print(historic_test_mags.shape)
print(historic_test_eventids.shape)
n_channels_hist = 3

bad_hist_indices = np.loadtxt('/home/sdybing/mlaapde/codes/bad_historic_waves.txt').astype(int)

historic_test_waves_nonan = np.delete(historic_test_waves, bad_hist_indices, axis = 0)
historic_test_mags_nonan = np.delete(historic_test_mags, bad_hist_indices, axis = 0)
historic_test_eventids_nonan = np.delete(historic_test_eventids, bad_hist_indices, axis = 0)

print(historic_test_waves.shape)
print(historic_test_waves_nonan.shape)
print(historic_test_mags.shape)
print(historic_test_mags_nonan.shape)
print(historic_test_eventids.shape)
print(historic_test_eventids_nonan.shape)

n_historic_test_samp = len(historic_test_waves_nonan)

historic_test_mags_nonan = np.round_(historic_test_mags_nonan, decimals = 1)

(13449, 2400, 3)
(13449,)
(13449,)
(13449, 2400, 3)
(13151, 2400, 3)
(13449,)
(13151,)
(13449,)
(13151,)


In [30]:
all_eventids = np.concatenate((np.array(mlaa_eventids), historic_test_eventids_nonan))
all_eventids.shape

(337516,)

In [64]:
all_test_preds = np.load('/home/sdybing/mlaapde/testdata_preds/all_test_rerun/mag_preds/114s_window_magpreds.npy')
all_test_preds.reshape((len(all_test_preds),)).shape

(337516,)

In [65]:
catalog_mags = np.concatenate((test_mags, historic_test_mags_nonan))
catalog_mags = np.round_(catalog_mags, decimals = 1)
catalog_mags.shape

(337516,)

In [66]:
measured_mags = catalog_mags
predicted_mags = all_test_preds.reshape((len(all_test_preds),))

In [158]:
tsuQ_1 = 'usp000hnj4' # Mentawai 7.8
tsuQ_2 = 'usp000ensm' # Pangandaran 7.7
deep_1 = 'usb000h4jh' # 598 km 8.3
deep_2 = 'usp000hgmq' # 641 km 7.5
deep_3 = 'us6000bi4p' # 624 km 6.9 (2020)
deep_4 = 'usp000hgmh' # 578 km 7.6, associated with deep_2
#deep_3 = 'usp000hgmh' # 578 km 7.6
#deep_4 = 'usp000ba6m' # 675 km 7.7
#deep_4 = 'usp000gbc0' # 633 km 7.7
complex_1 = 'official20120411083836720_20' # Wharton basin 8.6
complex_2 = 'us6000c9hg' # Sand Point 7.6 (2020)
# Kaikoura not in 2020 testing dataset or historic data

In [181]:
events = all_eventids
edge_idxs = []
normal_idxs = []

for idx in range(len(events)):
    
#     if events[idx] == deep_2 or events[idx] == deep_4: # JUST THE SUPPLEMENT ONES ### ----- FIGURE S7 ----- ###
    if events[idx] == tsuQ_1 or events[idx] == tsuQ_2 or events[idx] == deep_1 or events[idx] == deep_2 or events[idx] == deep_3 or events[idx] == complex_1 or events[idx] == complex_2: ### ----- FIGURE 9 ----- ###
        edge_idxs.append(idx)
    else:
        normal_idxs.append(idx)


In [182]:
len(edge_idxs)

1337

In [183]:
len(normal_idxs)

336179

In [185]:
cut_len = 120
shift_len = cut_len - 6

bins = np.arange(7,92,1)/10
normal_bins = []
edge_bins = []
all_bins = []
hist_bins = []

norm_all_meas_mags = measured_mags[normal_idxs]
edge_all_meas_mags = measured_mags[edge_idxs]

norm_all_pred_mags = predicted_mags[normal_idxs]
edge_all_pred_mags = predicted_mags[edge_idxs]

hist_meas_mags = measured_mags[-13151:]
hist_pred_mags = predicted_mags[-13151:]

for abin in bins:
#     print(abin)
    i = np.where(norm_all_meas_mags == abin)[0]
    j = np.where(edge_all_meas_mags == abin)[0]
    k = np.where(measured_mags == abin)[0]
    l = np.where(hist_meas_mags == abin)[0]
#     print(measured_mags[i])
    normal_predict_bin = np.array(norm_all_pred_mags[i])
    edge_predict_bin = np.array(edge_all_pred_mags[j])
    predict_bin = np.array(predicted_mags[k])
    hist_bin = np.array(hist_pred_mags[l])
#     print(predict_bin)
    normal_bins.append(normal_predict_bin)
    edge_bins.append(edge_predict_bin)
    all_bins.append(predict_bin)
    hist_bins.append(hist_bin)
#     print('-----')

# ----- Make boxplot ----- #

fig = plt.figure(figsize = (14, 9), dpi = 300, facecolor = 'white')

plt.rcParams['text.usetex'] = False

fig.suptitle(str(shift_len) + 's window', fontsize = 20, y = 0.93, color = 'black')
ax = fig.add_subplot(111)
ax.set_facecolor('white')
# ax.text(x = 30, y = 8.8, s = 'Model: ' + m_name, fontsize = 13, color = 'black')
ax.grid(which = 'major', axis = 'y', markevery = 0.5, zorder = 2)
ax.grid(which = 'major', axis = 'x', zorder = 2.5)
ax.set_ylim(0,9.2)

# bp = ax.boxplot(all_bins, notch = False, patch_artist = True, zorder = 3)

# for patch in bp['boxes']:
#     patch.set_facecolor('#72d1d6')
#     patch.set_edgecolor('#001528')
# for median in bp['medians']:
#     median.set(color = '#001528', linewidth = 3)
# for whisker in bp['whiskers']:
#     whisker.set(color = '#001528', linewidth = 1)
# for cap in bp['caps']:
#     cap.set(color = '#001528', linewidth = 1)
# for flier in bp['fliers']:
#     flier.set(marker = '+', color = '#001528', alpha = 0.5)

bp = ax.boxplot(normal_bins, notch = False, patch_artist = True, zorder = 3)

for patch in bp['boxes']:
    patch.set_facecolor('#72d1d6')
    patch.set_edgecolor('#001528')
for median in bp['medians']:
    median.set(color = '#001528', linewidth = 3)
for whisker in bp['whiskers']:
    whisker.set(color = '#001528', linewidth = 1)
for cap in bp['caps']:
    cap.set(color = '#001528', linewidth = 1)
for flier in bp['fliers']:
    flier.set(marker = '+', color = '#001528', alpha = 0.5)

bp2 = ax.boxplot(edge_bins, notch = False, patch_artist = True, zorder = 4)

for patch in bp2['boxes']:
    patch.set_facecolor('#f01f42')
    patch.set_edgecolor('#001528')
for median in bp2['medians']:
    median.set(color = '#001528', linewidth = 3)
for whisker in bp2['whiskers']:
    whisker.set(color = '#001528', linewidth = 1)
for cap in bp2['caps']:
    cap.set(color = '#001528', linewidth = 1)
for flier in bp2['fliers']:
    flier.set(marker = '+', color = '#001528', alpha = 0.5)
    
# bp3 = ax.boxplot(hist_bins, notch = False, patch_artist = True, zorder = 5)

# for patch in bp3['boxes']:
#     patch.set_facecolor('green')
#     patch.set_edgecolor('#001528')
# for median in bp3['medians']:
#     median.set(color = '#001528', linewidth = 3)
# for whisker in bp3['whiskers']:
#     whisker.set(color = '#001528', linewidth = 1)
# for cap in bp3['caps']:
#     cap.set(color = '#001528', linewidth = 1)
# for flier in bp3['fliers']:
#     flier.set(marker = '+', color = '#001528', alpha = 0.5)

# both_test_mags = [test_mags*10-6, historic_test_mags_nonan*10-6]
# ax2 = ax.twinx()
# ax2.hist(both_test_mags, bins = 85, color = ['#730114', '#f01f42'], stacked = True, zorder = 3.5)
# ax2.set_ylim(0,110000)
# ax2.set_yticks([0, 10000, 20000])
# ax2.set_yticklabels(['0', '10,000', '20,000'], style = 'italic')
# # ax2.set_ylabel('Count of waveforms in each magnitude bin', fontsize = 16, color = 'black', rotation = 270, labelpad = 18)
# ax2.text(s = 'Count of waveforms\nin each magnitude bin', x = 86.5, y = 26000, fontsize = 16, color = 'black', rotation = 270, style = 'italic')
# ax2.tick_params(labelsize = 14, color = 'black')
# ax2.set_zorder(3.5)

# ax2.axvline(((Mw*10)-6), color = 'blue', linestyle = '--', linewidth = 2, zorder = 20)

bins_list = bins.tolist()
ax.set_xticklabels(double_bins, fontsize = 16, color = 'black')
ax.xaxis.set_major_locator(ticker.FixedLocator([4, 14, 24, 34, 44, 54, 64, 74, 84]))
ax.set_yticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
ax.tick_params(labelsize = 16, color = 'black')
ax.set_ylabel('Predicted magnitude', fontsize = 18, color = 'black')
ax.set_xlabel('Catalog magnitude', fontsize = 18, color = 'black')

linex = [4, 14, 24, 34, 44, 54, 64, 74, 84]
liney = [1, 2, 3, 4, 5, 6, 7, 8, 9]
liney05u = [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5]
liney05d = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5]

ax.plot(linex, liney, 'black', linestyle = '--', linewidth = 3, alpha = 0.4)
ax.plot(linex, liney05u, 'gray', linestyle = '--', linewidth = 2, alpha = 0.4)
ax.plot(linex, liney05d, 'gray', linestyle = '--', linewidth = 2, alpha = 0.4)
# ax2.text(s = 'Testing data results', x = 2, y = 104000, fontsize = 18, backgroundcolor = '#72d1d6', color = 'black', zorder = 25)
# ax2.text(s = 'STF magnitude: ' + str(round(Mw,2)), x = 2, y = 96500, fontsize = 18, backgroundcolor = 'blue', color = 'white', zorder = 25)
# ax.text(s = 'MLAAPDE test data distrib.', x = 0, y = 0, fontsize = 18, backgroundcolor = '#730114', color = 'white', zorder = 25)
ax.text(s = 'Selected edge cases', x = 64, y = 0.3, fontsize = 18, backgroundcolor = '#f01f42', color = 'white', zorder = 25)

# plt.show();
#     plt.savefig(save_dir + '/boxplot_durline_' + m_name + '.png', format = 'PNG', facecolor = 'white', transparent = False)
#     plt.savefig('/home/sdybing/mlaapde/figures/testing/boxplots/' + str(cut_len-6) + 's_window.png', format = 'PNG', facecolor = 'white', transparent = False)
# plt.savefig(models_figs_path + 'boxplots/' + str(cut_len-6) + 's_window_boxplot_pred_vs_cat.png', format = 'PNG', facecolor = 'white', transparent = False)
plt.savefig('/home/sdybing/mlaapde/testdata_preds/all_test_rerun/edge_cases_v2_boxplot.png')
plt.close();


In [168]:
double_bins = bins_list + bins_list

In [130]:
bins = np.arange(7,92,1)/10
normal_bins = []
edge_bins = []
all_bins = []
hist_bins = []

hist_meas_mags = measured_mags[-13151:]
hist_pred_mags = predicted_mags[-13151:]

bins

array([0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9,
       2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2,
       3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5,
       4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8,
       5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7. , 7.1,
       7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8. , 8.1, 8.2, 8.3, 8.4,
       8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1])

In [131]:
print(measured_mags.shape)
print(predicted_mags.shape)

(337516,)
(337516,)


In [134]:
for abin in bins:
    print('-----------')
    print(abin)
#     i = np.where(measured_mags[normal_idxs] == abin)[0]
#     j = np.where(measured_mags[edge_idxs] == abin)[0]
    k = np.where(measured_mags == abin)[0] # Find the indices in the catalog mags where they match this bin
#     l = np.where(hist_meas_mags == abin)[0]
    print(measured_mags[k])
#     normal_predict_bin = np.array(predicted_mags[i])
#     edge_predict_bin = np.array(predicted_mags[j])
    predict_bin = np.array(predicted_mags[k]) # Get the predictions for those same bins
#     hist_bin = np.array(hist_pred_mags[l])
    print(predict_bin)
    normal_bins.append(normal_predict_bin)
    edge_bins.append(edge_predict_bin)
    all_bins.append(predict_bin)
    hist_bins.append(hist_bin)

-----------
0.7
[0.7 0.7 0.7 0.7 0.7 0.7]
[4.0147934 4.2549024 2.3469927 2.165854  4.5902963 3.7270546]
-----------
0.8
[0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8]
[2.0903635 2.8459284 3.8648994 4.4276023 2.1379032 2.1632986 3.2792952
 2.5716212 2.1255522 2.3370848 2.9919229 4.03311  ]
-----------
0.9
[0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9
 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9]
[2.2883797 2.3441174 3.3210251 2.4794    2.3254337 2.2041502 4.22575
 3.3429081 4.574386  2.1846354 2.1307948 2.8248966 3.5443878 2.613528
 2.7052336 3.3984604 3.1139834 2.0009582 3.0512688 4.346761  3.7392244
 3.8204722 2.5836058 2.6461625 2.3043284 2.4788346 4.269554  4.380024
 3.0556479]
-----------
1.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [135]:
norm_all_meas_mags = measured_mags[normal_idxs]
edge_all_meas_mags = measured_mags[edge_idxs]

norm_all_pred_mags = predicted_mags[normal_idxs]
edge_all_pred_mags = predicted_mags[edge_idxs]

In [138]:
print(len(norm_all_meas_mags))
print(len(norm_all_pred_mags))
print(len(edge_all_meas_mags))
print(len(edge_all_pred_mags))

335948
335948
1568
1568
